In [2]:
import pandas as pd
import numpy as np
import datetime as dt 


## Loading CSV and XLSX files (room price, last review and room type)


In [3]:
room_price = pd.read_csv(r"D:\Data Analyst projects\Nyc Analysis\airbnb_price.csv")
review = pd.read_csv(r"D:\Data Analyst projects\Nyc Analysis\airbnb_last_review.csv", sep = '\t')
type = pd.ExcelFile(r"D:\Data Analyst projects\Nyc Analysis\airbnb_room_type.xlsx")

In [4]:
#parsing the first sheet from type, room_types

room_type = type.parse()

In [6]:
room_price.head(10).style.background_gradient()

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"
5,5295,135 dollars,"Manhattan, Upper West Side"
6,5441,85 dollars,"Manhattan, Hell's Kitchen"
7,5803,89 dollars,"Brooklyn, South Slope"
8,6021,85 dollars,"Manhattan, Upper West Side"
9,6848,140 dollars,"Brooklyn, Williamsburg"


In [17]:
review.head().style.background_gradient()

,listing_id,host_name,last_review
0,2595,Jennifer,2019-05-21 00:00:00
1,3831,LisaRoxanne,2019-07-05 00:00:00
2,5099,Chris,2019-06-22 00:00:00
3,5178,Shunichi,2019-06-24 00:00:00
4,5238,Ben,2019-06-09 00:00:00


In [20]:
room_type.head().style.background_gradient()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,entire home/apt
1,3831,Cozy Entire Floor of Brownstone,entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,entire home/apt
5,5295,Beautiful 1br on Upper West Side,entire home/apt
6,5441,Central Manhattan/near Broadway,private room
7,5803,"Lovely Room 1, Garden, Best Area, Legal rental",private room
8,6021,Wonderful Guest Bedroom in Manhattan for SINGLES,private room
9,6848,Only 2 stops to Manhattan studio,entire home/apt


## Cleaning the Price Column to get the average price of room type

<p>You have noticed that the price <code>column</code> in the <code>room_price</code> DataFrame currently represents each value as a string with the currency (dollars)</p>

<p>We have to clean the data in order to calculate average price</p>

In [7]:
#Remove space and string characters from room price column
room_price["price"] = room_price["price"].astype(str)
room_price["price"] = room_price["price"].str.replace(" dollars", "")

# Convert prices column to numeric datatype
room_price["price"] = pd.to_numeric(room_price["price"])

# Print descriptive statistics for the price column
print(room_price["price"].describe())

count    25209.000000
mean       141.777936
std        147.349137
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max       7500.000000
Name: price, dtype: float64


## Calculating Average Price</p> 

<p>As we have seen, 75% of listings cost is $175(almost 3/4)</p>

<p>And there are some listings price which is  <code>$7500</code> and some of listings are for free <code>$0</code>.</p>

In [8]:
free_listings = room_price["price"] == 0
room_price = room_price.loc[~free_listings]
average_price = round(room_price['price'].mean(),2)
print("average_price per night of an Airbnb Listing is ${}.".format(average_price))

average_price per night of an Airbnb Listing is $141.82.


## Comparing Cost to the Private Rental Market

<p>After determining the average cost per night for a listing. According to <code>Zumper</code>, the average monthly rent for a 1-bedroom apartment in New York City is $3,100. To perform a meaningful comparison with the private market, we'll convert the <code>night_prices</code> of our listings into <code>monthly costs</code>.</p>

In [9]:
#Add new column as price per month
room_price["price_per_month"] = room_price["price"] * 365/12

#calculate average of price per month
avg_price_per_month = round(room_price["price_per_month"].mean(),2)

#Comparing Airbnb and Rental Market
print("Airbnb monthly price is ${}, and price of private rental is {}.".format(avg_price_per_month, "$3,100.00"))

Airbnb monthly price is $4313.61, and price of private rental is $3,100.00.


<p>Unsurprisingly, using Airbnb appears to be substantially more expensive than the private rental market. However, it's essential to consider that Airbnb listings encompass various accommodations, including single private rooms or even shared rooms, as well as entire homes/apartments.</p>

## Cleaning  Room Type Column

<p>In room type column there are some data which is not proper, some data is in lower case and some in upper case. We can standardize this by converting all string characters to lowercase</p>

In [10]:
#To Convert the room type column to lowercase
room_type["room_type"] = room_type["room_type"].str.lower()

#Upadte the room type column to cartegory data type
room_type["room_type"] = room_type["room_type"].astype("category")

#createing new variable as room frquencies
room_frequencies = room_type["room_type"].value_counts()

print(room_frequencies)

room_type
entire home/apt    13266
private room       11356
shared room          587
Name: count, dtype: int64


## What timeframe are we working with

The <code>last_review</code> column contains the date of the last review in the format of "Month Day Year" ( May 21 2019 ). We've been asked to find out the earliest and latest review dates in the DataFrame, and ensure the format allows this analysis to be easily conducted going forwards. </p>

In [11]:
#Change the data type of the last review column to datetime
review["last_review"] = pd.to_datetime(review["last_review"])

#Create first review , the earliest review date
first_review = review["last_review"].dt.date.min()

#Create last reviewed, the most recent review date
last_reviewed = review["last_review"].dt.date.max()

#Print the oldest and newest reviews from the dataset
print("The earliest Airbnb review is {}, the latest review is {}".format(first_review, last_reviewed))

The earliest Airbnb review is 2019-01-01, the latest review is 2019-07-09


## Joining the dataframe


<p>Now that we've extracted the necessary information, we will merge the three DataFrames to streamline any future analysis. After merging the data, we'll remove any observations with missing values and check for duplicates</p>

In [12]:
#Merging room price and room type to create new df rooms and prices
room_and_prices = room_price.merge(room_type, how="outer", on="listing_id")

#Merge price and type to create new df airbnb_merged
airbnb =  room_type.merge(room_and_prices, how="outer", on="listing_id")

#handling  missing values from airbnb
airbnb.dropna(inplace=True)

#Removing duplicate data
print("There are {} duplicates in the Dataframe.".format(airbnb.duplicated().sum()))

There are 0 duplicates in the Dataframe.


In [13]:
airbnb.head().style.background_gradient()

,listing_id,description_x,room_type_x,price,nbhood_full,price_per_month,description_y,room_type_y
0,2595,Skylit Midtown Castle,entire home/apt,225.000000,"Manhattan, Midtown",6843.750000,Skylit Midtown Castle,entire home/apt
1,3831,Cozy Entire Floor of Brownstone,entire home/apt,89.000000,"Brooklyn, Clinton Hill",2707.083333,Cozy Entire Floor of Brownstone,entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,entire home/apt,200.000000,"Manhattan, Murray Hill",6083.333333,Large Cozy 1 BR Apartment In Midtown East,entire home/apt
3,5178,Large Furnished Room Near B'way,private room,79.000000,"Manhattan, Hell's Kitchen",2402.916667,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,entire home/apt,150.000000,"Manhattan, Chinatown",4562.500000,Cute & Cozy Lower East Side 1 bdrm,entire home/apt


## Analyzing listing Prices by NYC borough

<p>Now that we have combined all data into a single DataFrame, Will focus on understanding the difference in listing prices between <code>NYC Boroughs</code>. 
Currently, boroughs are listed as the first part of a string within the <code>nbhood_full</code> column, for example:

<pre><code>Manhattan, Midtown
Brooklyn, Clinton Hill
Manhattan, Murray Hill
Manhattan, Hell's Kitchen
Manhattan, Chinatown</code></pre>

we need to extract this borough information from the string and store it in a new column, <code>borough</code>.</p>

In [14]:
# Extract data from the nbhood column and store as a new column, borough
airbnb["borough"] = airbnb["nbhood_full"].str.partition(",")[0]

#Grouping borough and calculating summary stat
boroughs = airbnb.groupby("borough")["price"].agg(["sum", "mean", "median", "count"])

# Round borough to 2 decimal places, and sort by mean in descending order
boroughs = boroughs.round(2).sort_values("mean", ascending=False)

print(boroughs)

                     sum    mean  median  count
borough                                        
Manhattan      1898963.0  184.03   149.0  10319
Brooklyn       1275435.0  122.02    95.0  10453
Queens          320765.0   92.81    70.0   3456
Staten Island    22974.0   86.04    71.0    267
Bronx            55231.0   79.24    65.0    697


## Price range by borough

<p>The above output provides a summary of prices for listings across the 5 boroughs. For this final task, we aim to categorize listings based on specific price ranges and view this breakdown by borough.</p>

In [15]:
#creating labels for the price range 
label_names = ["Budget", "Average", "Expensive", "Extravagant"]

#creating the label ranges
ranges = [0, 69, 175, 350, np.inf]

#Inserted new column, price range, into Dataframe
airbnb["price_range"] = pd.cut(airbnb["price"], bins=ranges, labels=label_names)

#calculate occurence frequencies for each label, prices_by_borough
price_by_borough = airbnb.groupby(["borough", "price_range"])["price_range"].count()
print(price_by_borough)

borough        price_range
Bronx          Budget          381
               Average         286
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5534
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5288
               Expensive      3073
               Extravagant     810
Queens         Budget         1632
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive        20
               Extravagant       0
Name: price_range, dtype: int64


C:\Users\lenovo\AppData\Local\Temp\ipykernel_13960\4038637146.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  price_by_borough = airbnb.groupby(["borough", "price_range"])["price_range"].count()
